<a href="https://colab.research.google.com/github/stormholman/Speech2Text/blob/main/s2t.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/openai/whisper.git 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-4q864osg
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-4q864osg
  Resolved https://github.com/openai/whisper.git to commit 248b6cb124225dd263bb9bd32d060b6517e067f8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.2 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=798075 sha256=f18d4dc465615cebd5cd997544baf81ce7a24acc362ebcbfc0b4cc71c834eb4f
  Stored in directory: /tmp/pip-ephem-wheel-cache-39_nr_um/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [ ]:
!sudo apt update && sudo apt install ffmpeg

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,344 kB]
Get:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2,270 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease


In [ ]:
!pip install kaggle
from google.colab import files
files.upload() # Upload kaggle.json

# Ensure kaggle.json is in the location ~/.kaggle/kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle competitions download -c ai-tech-challenge-2023
# Unzip the dataset
!unzip ai-tech-challenge-2023.zip -d dataset


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Streaming output truncated to the last 5000 lines.
  inflating: dataset/audio/audio/cv-valid-train/sample-190780.mp3  
  inflating: dataset/audio/audio/cv-valid-train/sample-190781.mp3  
  inflating: dataset/audio/audio/cv-valid-train/sample-190782.mp3  
  inflating: dataset/audio/audio/cv-valid-train/sample-190783.mp3  
  inflating: dataset/audio/audio/cv-valid-train/sample-190784.mp3  
  inflating: dataset/audio/audio/cv-valid-train/sample-190785.mp3  
  inflating: dataset/audio/audio/cv-valid-train/sample-190786.mp3  
  inflating: dataset/audio/audio/cv-valid-train/sample-190787.mp3  
  inflating: dataset/audio/audio/cv-valid-train/sample-190788.mp3  
  inflating: dataset/audio/audio/cv-valid-train/sample-190789.mp3  
  inflating: dataset/audio/audio/cv-valid-train/sample-190790.mp3  
  inflating: dataset/audio/audio/cv-valid-train/sample-190791.mp3  
  inflating: dataset/audio/audio/cv-valid-train/sample-190792.mp3  
  inflating: dataset/audio/audio/cv-valid-train/sample-190793.mp3

In [ ]:
# Now the transcription code
import os
import pandas as pd

folder_path = "/content/dataset/audio/audio/cv-valid-test"  # replace with the actual path to the mp3 files
file_list = os.listdir(folder_path)
file_list.sort()  # to ensure a consistent order across runs

# Load the last processed file index, or start from 0 if it doesn't exist
try:
    with open('last_processed.txt', 'r') as f:
        start_index = int(f.read().strip()) + 1
except FileNotFoundError:
    start_index = 0

transcriptions = {"ID": [], "transcription": []}

for index in range(start_index, len(file_list)):
    filename = file_list[index]
    if filename.endswith(".mp3"):
        filepath = os.path.join(folder_path, filename)
        # Now using the actual whisper command for ASR
        transcription = !whisper "{filepath}" --model medium
        transcription = transcription[-1].split('] ')[-1]  # extract the last part after '] '
        transcriptions["ID"].append(index)
        transcriptions["transcription"].append(transcription)

        # Save to CSV every 100 files, or if we've reached the last file
        if (index + 1) % 10 == 0 or index + 1 == len(file_list):
            df = pd.DataFrame(transcriptions)
            df.to_csv('a_transcriptions.csv', mode='a', header=(index == 0), index=False)
            transcriptions = {"ID": [], "transcription": []}  # reset for next batch
            with open('last_processed.txt', 'w') as f:
                f.write(str(index))  # save the index of the last processed file

Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: English
[00:00.000 --> 00:04.000]  Strange images pass through my mind.


In [ ]:
import os

old_file_name = "/content/transcriptions.csv"
new_file_name = "/content/a_transcriptions.csv"

os.rename(old_file_name, new_file_name)
